In [8]:
import os
from pyspark.sql import SparkSession
import requests
import pandas as pd
import numpy as np

# Set up request
def get_access_token():
    """Function to get access token from Barentswatch API

    Returns:
        str: Access token
    """
    url = "https://id.barentswatch.no/connect/token"
    # Read secret key from file
    secret_key = open(r'..\..\..\IND320\No_sync\fish_api', 'r').read()

    # Set up request to get access token
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "client_id": "erlend.risvik@gmail.com:fishclient",
        "scope": "api",
        "client_secret": secret_key,
        "grant_type": "client_credentials"
    }

    response = requests.post(url, headers=headers, data=data)
    return response.json()['access_token']

def convert_to_spark(df):
    """Function to convert pandas dataframe to spark dataframe

    Parameters:
    -----------
    df : pandas dataframe
        Dataframe to convert

    Returns:
    --------
    spark dataframe
    """
    return spark.createDataFrame(df)

def get_fish_data(year, week, access_token):
    """Function to get fish data from Barentswatch API.
    
    Parameters:
    -----------
    year : int
        Year of data
    week : int
        Week of data
    access_token : str
        Access token from Barentswatch API
    Returns:
    --------
    json: json object with data
    """

    # Set url to correct API address
    url = f"https://www.barentswatch.no/bwapi/v1/geodata/fishhealth/locality/{year}/{week}"

    headers = {
        "Authorization": "Bearer "+ access_token}

    df = requests.get(url, headers = headers).json()
    return df

def get_all_fish_data(year, access_token):
    """Function to get all fish data from Barentswatch API limited to one year.

    Parameters:
    -----------
    access_token : str
        Access token from Barentswatch API
    Returns:
    --------
    df: pandas dataframe with data
    """

    # Set list of weeks (1-52).
    weeks = np.arange(1, 53)
    df = pd.DataFrame()
    for week in weeks:
        data = get_fish_data(year = year, week = week, access_token = access_token)["localities"]
        data = pd.DataFrame(data)
        data["year"] = year
        data["week"] = week
        df = pd.concat([df, data], ignore_index=True)
    # Lowercase column names
    df.columns = df.columns.str.lower()
    return df


In [6]:

access_token = get_access_token()
test = get_fish_data(year = 2015, week = 1, access_token=access_token)
# print one sample of data
test['localities'][0]

{'localityNo': 14746,
 'localityWeekId': 713317,
 'name': 'Aarsand',
 'hasReportedLice': False,
 'isFallow': True,
 'avgAdultFemaleLice': None,
 'hasCleanerfishDeployed': False,
 'hasMechanicalRemoval': False,
 'hasSubstanceTreatments': False,
 'hasPd': False,
 'hasIla': False,
 'municipalityNo': '1811',
 'municipality': 'Bindal',
 'lat': 65.045867,
 'lon': 12.156933,
 'isOnLand': False,
 'inFilteredSelection': True,
 'hasSalmonoids': False,
 'isSlaughterHoldingCage': False}

In [9]:
test_2 = get_all_fish_data(year = 2015, access_token = access_token)

In [11]:
test_3 = get_all_fish_data(year = 2016, access_token = access_token)

In [12]:
test_2

,localityno,localityweekid,name,hasreportedlice,isfallow,avgadultfemalelice,hascleanerfishdeployed,hasmechanicalremoval,hassubstancetreatments,haspd,...,municipalityno,municipality,lat,lon,isonland,infilteredselection,hassalmonoids,isslaughterholdingcage,year,week
0,14746,713317,Aarsand,False,True,NaN,False,False,False,False,...,1811,Bindal,65.045867,12.156933,False,True,False,False,2015,1
1,12765,295408,Aasen Gård,False,True,NaN,False,False,False,False,...,0439,Folldal,62.093517,9.973667,True,True,True,False,2015,1
2,31937,717087,Abelsnes,False,True,NaN,False,False,False,False,...,1004,Flekkefjord,58.238767,6.656650,True,True,False,False,2015,1
3,10665,443570,Adamselv,False,True,NaN,False,False,False,False,...,2022,Lebesby,70.410660,26.687325,True,True,True,False,2015,1
4,29196,836973,Adjetjohka,False,True,NaN,False,False,False,False,...,2011,Guovdageaidnu-Kautokeino,68.944137,22.918715,True,True,False,False,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88659,24175,187315,Ytre Lavollsfjord,True,False,0.04,True,False,False,False,...,1929,Berg,69.419467,17.287567,False,True,True,False,2015,52
88660,20176,418617,Ytre Simanes,False,True,NaN,False,False,False,False,...,2012,Alta,69.948100,23.092333,False,True,True,True,2015,52
88661,11338,99369,Ytre Stræte,True,False,0.33,False,False,False,False,...,1913,Skånland,68.719483,17.138817,False,True,True,False,2015,52
88662,29417,231376,Ytre Vassholmen,True,False,0.17,False,False,False,False,...,1818,Herøy I Nordland,65.934967,12.139133,False,True,True,False,2015,52


In [14]:
test_3

,localityno,localityweekid,name,hasreportedlice,isfallow,avgadultfemalelice,hascleanerfishdeployed,hasmechanicalremoval,hassubstancetreatments,haspd,...,municipalityno,municipality,lat,lon,isonland,infilteredselection,hassalmonoids,isslaughterholdingcage,year,week
0,14746,713370,Aarsand,False,True,NaN,False,False,False,False,...,1811,Bindal,65.045867,12.156933,False,True,False,False,2016,1
1,12765,295461,Aasen Gård,False,True,NaN,False,False,False,False,...,0439,Folldal,62.093517,9.973667,True,True,True,False,2016,1
2,31937,717140,Abelsnes,False,True,NaN,False,False,False,False,...,1004,Flekkefjord,58.238767,6.656650,True,True,False,False,2016,1
3,10665,443623,Adamselv,False,True,NaN,False,False,False,False,...,2022,Lebesby,70.408000,26.691333,True,True,True,False,2016,1
4,29196,837026,Adjetjohka,False,True,NaN,False,False,False,False,...,2011,Guovdageaidnu-Kautokeino,68.944137,22.918715,True,True,False,False,2016,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87141,24175,277852,Ytre Lavollsfjord,True,False,0.02,False,False,False,False,...,1929,Berg,69.419467,17.287567,False,True,True,False,2016,52
87142,20176,418670,Ytre Simanes,False,True,NaN,False,False,False,False,...,2012,Alta,69.948100,23.092333,False,True,True,True,2016,52
87143,11338,277599,Ytre Stræte,True,False,0.18,False,False,False,False,...,1913,Skånland,68.719483,17.138817,False,True,True,False,2016,52
87144,29417,276893,Ytre Vassholmen,False,True,NaN,False,False,False,False,...,1818,Herøy I Nordland,65.934967,12.139133,False,True,True,False,2016,52
